# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="https://github.com/MyaMjechal/nlp-a6-lets-talk-with-yourself-rag-chatbot/blob/figures/RAG-process.png?raw=1" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [ ]:
# #langchain library
# !pip install langchain==0.0.350
# #LLM
# !pip install accelerate==0.25.0
# !pip install transformers==4.36.2
# !pip install bitsandbytes==0.41.2
# #Text Embedding
# !pip install sentence-transformers==2.2.2
# !pip install InstructorEmbedding==1.0.1
# #vectorstore
# !pip install pymupdf==1.23.8
# !pip install faiss-gpu==1.7.2
# !pip install faiss-cpu==1.7.4

In [2]:
!pip install langchain
!pip install accelerate
!pip install transformers
!pip install bitsandbytes
!pip install sentence-transformers
!pip install InstructorEmbedding
!pip install faiss-cpu
!pip install langchain-community
!pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00


In [1]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [2]:
from langchain import PromptTemplate

# prompt_template = """
#     I'm your friendly NLP chatbot named ChakyBot, here to assist Chaky and Gun with any questions they have about Natural Language Processing (NLP).
#     If you're curious about how probability works in the context of NLP, feel free to ask any questions you may have.
#     Whether it's about probabilistic models, language models, or any other related topic,
#     I'm here to help break down complex concepts into easy-to-understand explanations.
#     Just let me know what you're wondering about, and I'll do my best to guide you through it!
#     {context}
#     Question: {question}
#     Answer:
#     """.strip()

prompt_template = """
    I'm a friendly chatbot here to answer questions about Mya Mjechal myself based on my knowledge from my CV and personal data.
    The current year is 2025, and all answers should reflect this year unless otherwise specified.
    Whether you're curious about my education, work experience, or personal interests,
    I’ll provide accurate and gentle responses using the information I have.
    If I don’t know something, I’ll let you know kindly. Just ask away!
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="I'm a friendly chatbot here to answer questions about Mya Mjechal myself based on my knowledge from my CV and personal data. \n    The current year is 2025, and all answers should reflect this year unless otherwise specified. \n    Whether you're curious about my education, work experience, or personal interests, \n    I’ll provide accurate and gentle responses using the information I have. \n    If I don’t know something, I’ll let you know kindly. Just ask away!\n    {context}\n    Question: {question}\n    Answer:")

In [3]:
PROMPT.format(
    context = "My CV states that I graduated with a Bachelor’s degree in Computer Science from University of Information Technology.",
    question = "What is your highest level of education?"
)

"I'm a friendly chatbot here to answer questions about Mya Mjechal myself based on my knowledge from my CV and personal data. \n    The current year is 2025, and all answers should reflect this year unless otherwise specified. \n    Whether you're curious about my education, work experience, or personal interests, \n    I’ll provide accurate and gentle responses using the information I have. \n    If I don’t know something, I’ll let you know kindly. Just ask away!\n    My CV states that I graduated with a Bachelor’s degree in Computer Science from University of Information Technology.\n    Question: What is your highest level of education?\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code).
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [5]:
# from langchain.document_loaders import PyMuPDFLoader

# nlp_docs = '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf'

# loader = PyMuPDFLoader(nlp_docs)
# documents = loader.load()

from langchain.document_loaders import TextLoader
cv_file = 'data/MyaMjechal-CV.txt'
loader = TextLoader(cv_file)
documents = loader.load()

In [6]:
# documents

In [7]:
len(documents)

1

In [8]:
documents[0]

Document(metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='Mya Mjechal\nFull Stack Developer\n\nPathum Thani, Thailand\n+66823168693\nmyamjechal.mj@gmail.com\n\nBirthday: September 16, 1999\n\nI am a skilled developer with expertise in developing chatbots and dashboards, with additional skills in Natural Language Understanding (NLU) and Natural Language Processing (NLP) at Myanmar Youth Empowerment Opportunities (MYEO). I have experience as a freelancer, developing Odoo apps for local businesses, and a background in Software Development Management at Bliss Stock Co., Ltd. I am knowledgeable in Cluster Computing, Cloud Computing, Virtualization, Big Data Analysis, Networking, Web Programming, Blockchain, and IoT. I participated in the Huawei Cloud & AI contest, gaining experience in Data Mining, Image Processing, Data Prediction, and Cloud Usage. I implemented technologies in class projects, including OpenStack, Hyper-V Failover Clustering, Student Online Result System, and a

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [9]:
# # Remove newlines from the content
# for document in documents:
#     document.page_content = document.page_content.replace('\n', ', ').strip()

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [11]:
doc[1]

Document(metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='I am a skilled developer with expertise in developing chatbots and dashboards, with additional skills in Natural Language Understanding (NLU) and Natural Language Processing (NLP) at Myanmar Youth Empowerment Opportunities (MYEO). I have experience as a freelancer, developing Odoo apps for local businesses, and a background in Software Development Management at Bliss Stock Co., Ltd. I am knowledgeable in Cluster Computing, Cloud Computing, Virtualization, Big Data Analysis, Networking, Web Programming, Blockchain, and IoT. I participated in the Huawei Cloud & AI contest, gaining experience in Data Mining, Image Processing, Data Prediction, and Cloud Usage. I implemented technologies in')

In [12]:
len(doc)

8

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [13]:
import torch
# from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

<ipython-input-13-fba4864e35c7>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [14]:
#locate vectorstore
vector_path = '../vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

create path done


In [15]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'myamjechal_cv'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'cv' #from default index nlp
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [16]:
#calling vector from local
vector_path = '../vector-store'
db_file_name = 'myamjechal_cv'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'cv', #default index
    allow_dangerous_deserialization=True
)

In [17]:
#ready to use
retriever = vectordb.as_retriever()

In [18]:
# retriever.get_relevant_documents("How old are you?")
retriever.invoke("How old are you?")

[Document(id='3d958a58-f8a4-46e7-aaa0-0c9f1289eeb9', metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='I am a skilled developer with expertise in developing chatbots and dashboards, with additional skills in Natural Language Understanding (NLU) and Natural Language Processing (NLP) at Myanmar Youth Empowerment Opportunities (MYEO). I have experience as a freelancer, developing Odoo apps for local businesses, and a background in Software Development Management at Bliss Stock Co., Ltd. I am knowledgeable in Cluster Computing, Cloud Computing, Virtualization, Big Data Analysis, Networking, Web Programming, Blockchain, and IoT. I participated in the Huawei Cloud & AI contest, gaining experience in Data Mining, Image Processing, Data Prediction, and Cloud Usage. I implemented technologies in'),
 Document(id='944f1079-b7c6-4268-8832-d02e30cd40e8', metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='* June 2022 – December 2023\n  Computer Programmer | Myanmar Youth Empower

In [19]:
# retriever.get_relevant_documents("How many years of work experience do you have?")
retriever.invoke("How many years of work experience do you have?")

[Document(id='3d958a58-f8a4-46e7-aaa0-0c9f1289eeb9', metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='I am a skilled developer with expertise in developing chatbots and dashboards, with additional skills in Natural Language Understanding (NLU) and Natural Language Processing (NLP) at Myanmar Youth Empowerment Opportunities (MYEO). I have experience as a freelancer, developing Odoo apps for local businesses, and a background in Software Development Management at Bliss Stock Co., Ltd. I am knowledgeable in Cluster Computing, Cloud Computing, Virtualization, Big Data Analysis, Networking, Web Programming, Blockchain, and IoT. I participated in the Huawei Cloud & AI contest, gaining experience in Data Mining, Image Processing, Data Prediction, and Cloud Usage. I implemented technologies in'),
 Document(id='eb35ae01-be90-4543-8224-5e162d5c19bd', metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='WORK EXPERIENCE\n* January 2025 – Present\n  Technical Support | intERLab 

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [20]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

InMemoryChatMessageHistory(messages=[])

In [21]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [22]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Whats up?', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm quite good. How about you?", additional_kwargs={}, response_metadata={})])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios.
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [23]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

<ipython-input-23-caba3bb497d2>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [24]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content="What's up?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm quite good. How about you?", additional_kwargs={}, response_metadata={})]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time.
- it only uses the last K interactions.
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [25]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

<ipython-input-25-b8b41c031995>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available),
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [26]:
# %cd ./models
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

In [33]:
from huggingface_hub import login

login()

In [34]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
# from langchain import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline
import torch

# model_id = '../models/fastchat-t5-3b-v1.0/'
model_id = "google/gemma-2-9b"

tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto'
    # load_in_8bit = True
)

pipe = pipeline(
    # task="text2text-generation",
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0.1, # from 0
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/2.38G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

Device set to use cuda:0


In [35]:
# check_result_3 = pipe("What is the current year?")
# print(check_result_3)

In [36]:
# check_result = llm.invoke("Hi, how old am I now if I was born in 1999?")
# print(check_result)

In [37]:
# check_result_1 = llm.invoke("What is the current year?")
# print(check_result_1)

### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [38]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [48]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], input_types={}, partial_variables={}, template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [51]:
# question_generator = LLMChain(
#     llm = llm,
#     prompt = CONDENSE_QUESTION_PROMPT,
#     verbose = True
# )
question_generator = CONDENSE_QUESTION_PROMPT | llm

In [54]:
# query = 'Since when?'
# chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

# question_generator({'chat_history' : chat_history, "question" : query})

query = "Since when?"
# chat_history = "Human: How old are you?\nAI:\nHuman: Where do you work now?\nAI:"
chat_history = """
Human: How old are you?
AI: I was born on September 16, 1999, so I am 26 years old in 2025.
Human: Where do you work now?
AI: I currently work as a Technical Support at intERLab, Asian Institute of Technology, starting January 2025.
"""

question_generator.invoke({'chat_history': chat_history, 'question': query})

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


'Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n\nHuman: How old are you?\nAI: I was born on September 16, 1999, so I am 26 years old in 2025.\nHuman: Where do you work now?\nAI: I currently work as a Technical Support at intERLab, Asian Institute of Technology, starting January 2025.\n\nFollow Up Input: Since when?\nStandalone question: Since when do you work at intERLab?'

`combine_docs_chain`

In [56]:
from langchain.chains import ConversationalRetrievalChain, StuffDocumentsChain

# doc_chain = load_qa_chain(
#     llm = llm,
#     chain_type = 'stuff',
#     prompt = PROMPT,
#     verbose = True
# )
# doc_chain
doc_chain = StuffDocumentsChain(
    llm_chain=LLMChain(llm=llm, prompt=PROMPT),
    document_variable_name="context",
    verbose=True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="I'm a friendly chatbot here to answer questions about Mya Mjechal myself based on my knowledge from my CV and personal data. \n    The current year is 2025, and all answers should reflect this year unless otherwise specified. \n    Whether you're curious about my education, work experience, or personal interests, \n    I’ll provide accurate and gentle responses using the information I have. \n    If I don’t know something, I’ll let you know kindly. Just ask away!\n    {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7e14fbe6db10>, model_id='google/gemma-2-9b'), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_vari

In [58]:
# query = "What is Transformers?"
# input_document = retriever.get_relevant_documents(query)

# doc_chain({'input_documents':input_document, 'question':query})

query = "How old are you?"
input_documents = retriever.invoke(query)
doc_chain.invoke({'input_documents': input_documents, 'question': query})

{'input_documents': [Document(id='3d958a58-f8a4-46e7-aaa0-0c9f1289eeb9', metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='I am a skilled developer with expertise in developing chatbots and dashboards, with additional skills in Natural Language Understanding (NLU) and Natural Language Processing (NLP) at Myanmar Youth Empowerment Opportunities (MYEO). I have experience as a freelancer, developing Odoo apps for local businesses, and a background in Software Development Management at Bliss Stock Co., Ltd. I am knowledgeable in Cluster Computing, Cloud Computing, Virtualization, Big Data Analysis, Networking, Web Programming, Blockchain, and IoT. I participated in the Huawei Cloud & AI contest, gaining experience in Data Mining, Image Processing, Data Prediction, and Cloud Usage. I implemented technologies in'),
  Document(id='944f1079-b7c6-4268-8832-d02e30cd40e8', metadata={'source': 'data/MyaMjechal-CV.txt'}, page_content='* June 2022 – December 2023\n  Computer Programmer | 

In [61]:
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

# chain = ConversationalRetrievalChain(
#     retriever=retriever,
#     question_generator=question_generator,
#     combine_docs_chain=doc_chain,
#     return_source_documents=True,
#     memory=memory,
#     verbose=True,
#     get_chat_history=lambda h : h
# )
# chain

from langchain_core.runnables import RunnableLambda

# Wrap question_generator to mimic LLMChain behavior
def run_question_generator(inputs):
    return question_generator.invoke(inputs)

question_generator_chain = RunnableLambda(run_question_generator)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    combine_docs_chain=doc_chain,  # Only pass doc_chain here
    memory=memory,
    verbose=True,
    question_generator=question_generator,
    return_source_documents=True,
    get_chat_history=lambda h: h
)
chain

TypeError: langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain() got multiple values for keyword argument 'combine_docs_chain'

## 5. Chatbot

In [45]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer

{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': "I'm a friendly chatbot here to answer questions about Mya Mjechal myself based on my knowledge from my CV and personal data. \n    The current year is 2025, and all answers should reflect this year unless otherwise specified. \n    Whether you're curious about my education, work experience, or personal interests, \n    I’ll provide accurate and gentle responses using the information I have. \n    If I don’t know something, I’ll let you know kindly. Just ask away!\n    I am a skilled developer with expertise in developing chatbots and dashboards, with additional skills in Natural Language Understanding (NLU) and Natural Language Processing (NLP) at Myanmar Youth Empowerment Opportunities (MYEO). I have experience as a freelancer, developing Odoo apps for local businesses, and a background in Software Development Management at Bliss Stock Co., Ltd. I am knowledgeable in Cluster Computing, Cloud Computing, Virtualiza

In [46]:
prompt_question = "What is the Transformers?"
answer = chain({"question":prompt_question})
answer

{'question': 'What is the Transformers?',
 'chat_history': [HumanMessage(content='Who are you by the way?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm a friendly chatbot here to answer questions about Mya Mjechal myself based on my knowledge from my CV and personal data. \n    The current year is 2025, and all answers should reflect this year unless otherwise specified. \n    Whether you're curious about my education, work experience, or personal interests, \n    I’ll provide accurate and gentle responses using the information I have. \n    If I don’t know something, I’ll let you know kindly. Just ask away!\n    I am a skilled developer with expertise in developing chatbots and dashboards, with additional skills in Natural Language Understanding (NLU) and Natural Language Processing (NLP) at Myanmar Youth Empowerment Opportunities (MYEO). I have experience as a freelancer, developing Odoo apps for local businesses, and a background in Software Development Man

In [47]:
prompt_question = "Is it a statistical model?"
answer = chain({"question":prompt_question})
answer

{'question': 'Is it a statistical model?',
 'chat_history': [HumanMessage(content='Who are you by the way?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm a friendly chatbot here to answer questions about Mya Mjechal myself based on my knowledge from my CV and personal data. \n    The current year is 2025, and all answers should reflect this year unless otherwise specified. \n    Whether you're curious about my education, work experience, or personal interests, \n    I’ll provide accurate and gentle responses using the information I have. \n    If I don’t know something, I’ll let you know kindly. Just ask away!\n    I am a skilled developer with expertise in developing chatbots and dashboards, with additional skills in Natural Language Understanding (NLU) and Natural Language Processing (NLP) at Myanmar Youth Empowerment Opportunities (MYEO). I have experience as a freelancer, developing Odoo apps for local businesses, and a background in Software Development Ma